In [50]:
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2

In [51]:
yaml_path = os.path.join(os.getcwd(), 'data-yolo', 'data.yaml')

In [52]:
def train_yolov11(yaml_path, epochs=30, batch_size=16, pretrained_model=None, img_size=416, learning_rate=0.0008, patience=7, device=0):
    """
    Train YOLOv11 on the prepared dataset, optionally starting from a pretrained model.
    
    Args:
        yaml_path (str): Path to the data.yaml file.
        epochs (int): Number of training epochs.
        batch_size (int): Batch size for training.
        pretrained_model (str): Path to a pretrained model to continue training. Default is None.
        img_size (int): Image size for training. Default is 416.
        learning_rate (float): Initial learning rate. Default is 0.0008.
        optimizer (str): Optimizer to use. Default is 'Adam'.
        patience (int): Number of epochs with no improvement after which training will be stopped. Default is 7.
        device (int): Device to use for training. Default is 0 (GPU).
    """
    # Ensure data.yaml exists
    if not os.path.exists(yaml_path):
        raise FileNotFoundError(f"data.yaml file not found: {yaml_path}")

    # Debug: Check if the dataset paths are correct
    print(f"Training with dataset located at: {yaml_path}")
    
    # Load YOLOv11 model (optionally using a pretrained model)
    model = YOLO(pretrained_model if pretrained_model else "yolo11s.pt")

    # Debug: Verify the model is loaded
    print(f"Model {pretrained_model if pretrained_model else 'yolo11s.pt'} loaded successfully.")
    
    # Train the model
    model.train(
        data=yaml_path,
        epochs=epochs,               # Use the provided epochs parameter
        batch=batch_size,            # Use the provided batch_size parameter
        imgsz=img_size,              # Use the provided img_size parameter
        lr0=learning_rate,           # Use the provided learning_rate parameter
        lrf=0.000001,                # Learning rate decay factor
        patience=patience,           # Use the provided patience parameter
        val=True,
        project="runs",
        name="yolov11_garbage_optimized",
        plots=True,
        device=device              # Use the provided device parameter
    )

    print("YOLOv11 training complete.")
    
    # Save the model after training
    model.save("yolov11_garbage_detection_model.pt")
    print("Model saved as 'yolov11_garbage_detection_model.pt'.")

In [53]:
def evaluate_model(model, yaml_path):
    """
    Evaluate the model using mAP and TensorBoard.

    Args:
        model: Trained YOLOv11 model.
        dataset (dict): Dictionary containing dataset paths.

    Returns:
        dict: Evaluation metrics including mAP scores.
    """
    print("Evaluating model performance...")

    # Run validation
    results = model.val(
        data=yaml_path,
        project="runs",
        name="evaluation YOLOv11 Model",
        imgsz=480,
        split='test',
        save_json=True,
        plots=True,
        exist_ok=True  # This will overwrite the existing evaluation folder
    )

    # Extract metrics safely
    try:
        metrics = {
            "mAP@0.5": results.box.map50,      # mAP@0.5
            "mAP@0.5:0.95": results.box.map    # mAP@0.5:0.95
        }
        print(f"Evaluation metrics: {metrics}")
        return metrics
    except AttributeError as e:
        raise ValueError("The results object does not contain expected metrics.") from e

In [54]:
# Train the model
train_yolov11(
    yaml_path=yaml_path,
    epochs=250,
    batch_size=16,
    pretrained_model=None,
    img_size=480,
    learning_rate=0.0008,
    patience=20,
    device=0
)

# Load the trained model
model = YOLO("yolov11_garbage_detection_model.pt")

# Evaluate model performance
metrics = evaluate_model(model, yaml_path)

Training with dataset located at: c:\Users\User\Desktop\Garbage-Classification\data-yolo\data.yaml
Model yolo11s.pt loaded successfully.
New https://pypi.org/project/ultralytics/8.3.68 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.67  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=c:\Users\User\Desktop\Garbage-Classification\data-yolo\data.yaml, epochs=250, time=None, patience=20, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs, name=yolov11_garbage_optimized5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=

train: Scanning C:\Users\User\Desktop\Garbage-Classification\data-yolo\train\labels.cache... 711 images, 0 backgrounds, 0 corrupt: 100%|██████████| 711/711 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 6, len(boxes) = 1610. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning C:\Users\User\Desktop\Garbage-Classification\data-yolo\valid\labels.cache... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<?, ?it/s]


Plotting labels to runs\yolov11_garbage_optimized5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0008' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)


...


108 epochs completed in 0.501 hours.
Optimizer stripped from runs\yolov11_garbage_optimized5\weights\last.pt, 19.2MB
Optimizer stripped from runs\yolov11_garbage_optimized5\weights\best.pt, 19.2MB

Validating runs\yolov11_garbage_optimized5\weights\best.pt...
Ultralytics 8.3.67  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


                   all         30         78       0.92      0.785      0.869      0.678
           Mixed Waste         10         19      0.978      0.947      0.993      0.794
         Organic Waste          9         11      0.965      0.909      0.988      0.833
           Other Waste         10         16      0.817      0.562       0.66      0.413
   Recyclable Material         12         32       0.92      0.722      0.835      0.672
Speed: 0.5ms preprocess, 10.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\yolov11_garbage_optimized5
YOLOv11 training complete.
Model saved as 'yolov11_garbage_detection_model.pt'.
Evaluating model performance...
Ultralytics 8.3.67  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
YOLO11s summary (fused): 238 layers, 9,414,348 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning C:\Users\User\Desktop\Garbage-Classification\data-yolo\test\labels.cache... 30 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30/30 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.29s/it]


                   all         30         91      0.787       0.63      0.725      0.511
           Mixed Waste          9         16      0.818      0.812      0.844      0.618
         Organic Waste         11         39      0.923      0.612      0.853      0.563
           Other Waste          9         17      0.463      0.205      0.262      0.154
   Recyclable Material         10         19      0.944       0.89      0.943      0.708
Speed: 1.2ms preprocess, 14.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Saving runs\evaluation YOLOv11 Model\predictions.json...
Results saved to runs\evaluation YOLOv11 Model
Evaluation metrics: {'mAP@0.5': 0.7254625514661821, 'mAP@0.5:0.95': 0.5109757002971055}
